## 6. Metaclasses and Attributes

### 44 Use Plain Attributes Instead of Setter and Getter Methods

In [1]:
import logging

In [2]:
class OldResistor:
    def __init__(self, ohms):
        self._ohms = ohms

    def get_ohms(self):
        return self._ohms

    def set_ohms(self, ohms):
        self._ohms = ohms

r0 = OldResistor(50e3)
print('Before:', r0.get_ohms())
r0.set_ohms(10e3)
print('After: ', r0.get_ohms())

r0.set_ohms(r0.get_ohms() - 4e3)
assert r0.get_ohms() == 6e3

Before: 50000.0
After:  10000.0


In [3]:
class Resistor:
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0

r1 = Resistor(50e3)
r1.ohms = 10e3
print(f'{r1.ohms} ohms, '
      f'{r1.voltage} volts, '
      f'{r1.current} amps')

r1.ohms += 5e3
assert r1.ohms == 15e3

10000.0 ohms, 0 volts, 0 amps


In [4]:
class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0

    @property
    def voltage(self):
        return self._voltage

    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms

r2 = VoltageResistance(1e3)
print(f'Before: {r2.current:.2f} amps')
r2.voltage = 10
print(f'After:  {r2.current:.2f} amps')

Before: 0.00 amps
After:  0.01 amps


In [5]:
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)

    @property
    def ohms(self):
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError(f'ohms must be > 0; got {ohms}')
        self._ohms = ohms

In [6]:
try:
    r3 = BoundedResistance(1e3)
    r3.ohms = 0
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-6-500ef67ee041>", line 3, in <module>
    r3.ohms = 0
  File "<ipython-input-5-71975c648873>", line 12, in ohms
    raise ValueError(f'ohms must be > 0; got {ohms}')
ValueError: ohms must be > 0; got 0


In [7]:
try:
    BoundedResistance(-5)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-7-693d2cd5f3c8>", line 2, in <module>
    BoundedResistance(-5)
  File "<ipython-input-5-71975c648873>", line 3, in __init__
    super().__init__(ohms)
  File "<ipython-input-3-dba1cbccceda>", line 3, in __init__
    self.ohms = ohms
  File "<ipython-input-5-71975c648873>", line 12, in ohms
    raise ValueError(f'ohms must be > 0; got {ohms}')
ValueError: ohms must be > 0; got -5


In [8]:
class FixedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)

    @property
    def ohms(self):
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, '_ohms'):
            raise AttributeError("Ohms is immutable")
        self._ohms = ohms

try:
    r4 = FixedResistance(1e3)
    r4.ohms = 2e3
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-8-7dc329b93c33>", line 17, in <module>
    r4.ohms = 2e3
  File "<ipython-input-8-7dc329b93c33>", line 12, in ohms
    raise AttributeError("Ohms is immutable")
AttributeError: Ohms is immutable


In [9]:
class MysteriousResistor(Resistor):
    @property
    def ohms(self):
        self.voltage = self._ohms * self.current
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        self._ohms = ohms

r7 = MysteriousResistor(10)
r7.current = 0.01
print(f'Before: {r7.voltage:.2f}')
r7.ohms
print(f'After:  {r7.voltage:.2f}')

Before: 0.00
After:  0.10


> - 새로운 클래스 인터페이스를 정의할 때는 간단하게 공개 애트리뷰트에서 시작하고, 세터나 게터 메서드를 가급적 사용하지 않는다.
> - 객체에 있는 애트리뷰트에 접근할 때 특별한 동작이 필요하면 `@property`로 이를 구현할 수 있다.
> - `@property` 메서드를 만들 때는 최소 놀람의 법칙을 따르고 이상한 부작용을 만들어내지 말라.
> - `@property` 메서드가 빠르게 실행되도록 유지하라. 느리거나 복잡한 작업의 경우(특히 I/O를 수행하는 등의 부수 효과가 있는 경우)에는 프로퍼티 대신 일반적인 메서드를 사용하라.